In [5]:
import random
import copy
import numpy as np
import pyspiel
from open_spiel.python.algorithms import mcts
from othello_game import OthelloGame, OthelloState
from MCTS import MCTS
from NNet import NNet

In [71]:
game = pyspiel.load_game("othello_nxn")
state = game.new_initial_state()
initial_state = copy.deepcopy(state)

nnet = NNet()
mcts = MCTS(game, nnet, numMCTSSims=500)

# mcts.getActionProb(state)

while not state.is_terminal():
    if state.current_player() == 0:
        probs = mcts.getActionProb(state)
        state.apply_action(np.argmax(probs))
        # state.apply_action(np.argmin(probs))
        # print("Es")
        # for key, value in mcts.Es.items():
        #     if value != 0:
        #         print(key)
        #         print(value)
        # print("Qsa")
        # for key, value in mcts.Qsa.items():
        #     print(key[0])
        #     print(key[1])
        #     print(value)
        # break
    else:
        action = random.choice(state.legal_actions())
        state.apply_action(action)

print(state.returns())
# print(np.array(state.observation_tensor(0)).reshape((2, 4, 4)))

[1.0, -1.0]


In [60]:
# s = """. . . .
# . O X .
# . X O .
# . . . ."""

s = str(initial_state)

children = mcts.tree[s]

while len(children) > 0:
    next_state, action = random.choice(list(children))
    print(next_state)
    print(action)
    print(mcts.Qsa[(s, action)])
    if next_state in mcts.tree:
        children = mcts.tree[next_state]
    else:
        children = []
    s = next_state

. . . . . .
. . . . . .
. . O X . .
. . X X . .
. . . X . .
. . . . . .
27
0.04188034188034187
. . . . . .
. . . . . .
. . O X . .
. . X O . .
. . . X O .
. . . . . .
28
-0.047222222222222214
. . . . . .
. . . . . .
. . O X . .
. . X O . .
. . . X X X
. . . . . .
29
0.06313131313131314
. . . . . .
. . . . . .
. . O X . .
. . X O . .
. . . X O X
. . . . . O
35
-0.0


In [3]:
np.array(state.observation_tensor(0)).reshape(2, 4, 4)

array([[[0., 1., 0., 0.],
        [0., 1., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 0.]]])